In [1]:
from keras.models import Sequential
from keras import layers

import re
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

df = pd.read_csv('../input/fake-news/train.csv')
test = pd.read_csv('../input/fake-news/test.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df = df.fillna('')
test = test.fillna('')
df['total'] = df['author']+' '+df['title']
test['total']=test['author']+' '+test['title']

In [3]:
X = df.drop('label',axis=1)
y = df['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


In [4]:
def stemmer(input_):
    review = re.sub('[^a-zA-Z]',' ',input_)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    return review

df['total'] = df['total'].apply(stemmer)

In [5]:
sentences = df['total'].values
labels = df['label'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.25, random_state=1000)

In [6]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

# len(vectorizer.vocabulary_) = 15031

In [7]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                150320    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 150,331
Trainable params: 150,331
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(X_train, y_train,
                    epochs=6,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/6
1560/1560 [==============================] - 3s 2ms/step - loss: 0.1537 - accuracy: 0.9647 - val_loss: 0.0498 - val_accuracy: 0.9887
Epoch 2/6
1560/1560 [==============================] - 3s 2ms/step - loss: 0.0183 - accuracy: 0.9962 - val_loss: 0.0317 - val_accuracy: 0.9919
Epoch 3/6
1560/1560 [==============================] - 3s 2ms/step - loss: 0.0050 - accuracy: 0.9995 - val_loss: 0.0275 - val_accuracy: 0.9931
Epoch 4/6
1560/1560 [==============================] - 3s 2ms/step - loss: 0.0017 - accuracy: 0.9999 - val_loss: 0.0272 - val_accuracy: 0.9919
Epoch 5/6
1560/1560 [==============================] - 3s 2ms/step - loss: 6.5437e-04 - accuracy: 1.0000 - val_loss: 0.0269 - val_accuracy: 0.9921
Epoch 6/6
1560/1560 [==============================] - 3s 2ms/step - loss: 2.7004e-04 - accuracy: 1.0000 - val_loss: 0.0290 - val_accuracy: 0.9925


In [9]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 1.0000
Testing Accuracy:  0.9925


In [10]:
model.save('.')